In [1]:
import os
import keras
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from skimage import transform 

from PIL import Image

import keras as K

In [2]:
main_folder = r'.\Skany'

images_dict = {}
labels_dict = {}

images_list_arrays = []
label_list_arrays = []

for subfolder in os.listdir(main_folder):   
    subfolder_path = os.path.join(main_folder, subfolder)

    for folder in os.listdir(subfolder_path):
        folder_path = os.path.join(subfolder_path, folder)

        if (folder_path[-4:]) == '.txt':
            f = open(folder_path, "r", encoding='ISO-8859-2')
            label = int(f.readlines()[1][7]) - 1
        elif (folder_path[-4:]) == '.jpg':
            continue
        else:
            images_list = []
            for image_path in os.listdir(folder_path):
                with Image.open(os.path.join(folder_path, image_path)) as img:
                    img = img.convert('RGB')
                    img = transform.resize(np.array(img), (224, 224), anti_aliasing=False)
                    images_list.append(img)


            #print(folder_path.split('\\')[-1], np.concatenate(images_list).shape)
            if np.concatenate(images_list).shape != (1792, 224, 3):
                pass
            else:
                images_dict[folder_path.split('\\')[-1]] = np.concatenate(images_list)
                images_list_arrays.append(np.concatenate(images_list))


                labels_dict[folder_path.split('\\')[-1]] = label
                label_list_arrays.append(label)


In [3]:
# main_folder = r'.\Skany'

# images_dict = {}
# labels_dict = {}

# images_list_arrays = []
# label_list_arrays = []

# for subfolder in os.listdir(main_folder):   
#     subfolder_path = os.path.join(main_folder, subfolder)

#     for folder in os.listdir(subfolder_path):
#         folder_path = os.path.join(subfolder_path, folder)

#         if (folder_path[-4:]) == '.txt':
#             f = open(folder_path, "r", encoding='ISO-8859-2')
#             label = int(f.readlines()[1][7]) - 1
#         elif (folder_path[-4:]) == '.jpg':
#             continue
#         else:
#             images_list = []
#             for image_path in os.listdir(folder_path):
#                 with Image.open(os.path.join(folder_path, image_path)) as img:
#                     img = img.convert('RGB')
#                     img = transform.resize(np.array(img), (224, 224), anti_aliasing=False)
#                     images_list.append(img)
#                 break

#             images_dict[folder_path.split('\\')[-1]] = (img)
#             images_list_arrays.append((img))


#             labels_dict[folder_path.split('\\')[-1]] = label
#             label_list_arrays.append(label)


In [4]:
# images_dict[folder_path.split('\\')[-1]] = np.concatenate(images_list)
#             images_list_arrays.append(np.concatenate(images_list))


In [5]:
len(images_list_arrays)

120

In [6]:
numpy_images = np.stack(images_list_arrays, axis = 0)
numpy_labels = np.stack(label_list_arrays, axis = 0)


In [7]:
numpy_labels = numpy_labels.reshape((numpy_labels.shape[0], 1))

In [8]:
###########

num_of_classes = 5

input_t = K.Input(shape = (numpy_images.shape[1], numpy_images.shape[2], 3))

###########

czy ten układ 1792, 224, 3 (224x224 8 obrazków najebanych na kupe) jest sensowny?

In [9]:
numpy_images.shape, numpy_labels.shape

((120, 1792, 224, 3), (120, 1))

In [10]:
numpy_labels.shape

(120, 1)

In [11]:
x= preprocess_input(numpy_images)


In [12]:
x.shape

(120, 1792, 224, 3)

In [13]:
# numpy_images = np.transpose(numpy_images, (0, 3, 1, 2))

In [14]:
numpy_images.shape

(120, 1792, 224, 3)

In [15]:
from tensorflow.keras.layers import Flatten, Dense, GlobalAveragePooling2D



In [16]:
res_model = keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=input_t,
    classes=num_of_classes,
    pooling = 'avg',
    classifier_activation="softmax",
)


można sprawdzić czy nie będzie lepiej działać jak utniemy więcej warst (jak tu jest ze 150 to zamiast zostawiać wszsytkie 150, to zostawimy pierwsze x)

In [17]:
for layer in res_model.layers:
    layer.trainable = False

In [18]:
from tensorflow.keras.utils import to_categorical

In [19]:
y = to_categorical(numpy_labels, num_of_classes)

In [20]:
numpy_labels.shape

(120, 1)

In [26]:
from tensorflow.keras.metrics import categorical_crossentropy, CategoricalAccuracy

czy my nie chcemy tutaj dodać może jakiejś swojej ekstra warstwy?

dobrze by było coś podziałać z dystrybucją - jakieś weighted categorical cross entropy??

In [27]:
model = K.models.Sequential()
model.add(res_model)
model.add(K.layers.Flatten())
model.add(K.layers.Dense(num_of_classes, activation='softmax'))

model.compile(optimizer=keras.optimizers.Adam(),  # Very low learning rate
              loss="categorical_crossentropy",
              metrics=[CategoricalAccuracy()])

# Train end-to-end. Be careful to stop before you overfit!
model.fit(x=x, y=y, batch_size = 16, epochs=5)


Epoch 1/5
8/8 [==============================] - 41s 5s/step - loss: 1.4270 - categorical_accuracy: 0.3583
Epoch 2/5
8/8 [==============================] - 37s 5s/step - loss: 1.3072 - categorical_accuracy: 0.3333
Epoch 3/5
8/8 [==============================] - 36s 4s/step - loss: 1.2848 - categorical_accuracy: 0.4000
Epoch 4/5
8/8 [==============================] - 36s 4s/step - loss: 1.2774 - categorical_accuracy: 0.4000
Epoch 5/5
8/8 [==============================] - 35s 4s/step - loss: 1.2900 - categorical_accuracy: 0.3500


In [28]:
model.evaluate(x=x, y=y)


4/4 [==============================] - 36s 9s/step - loss: 1.2567 - categorical_accuracy: 0.4250


[1.2567368745803833, 0.42500001192092896]